In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
sys.path.append("../..")


import pandas as pd
from typing import Dict, List
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns
import numpy as np
import json

sns.set_style("whitegrid", {"grid.color": ".93"})

from src.paper_analysis_hydropower import NationalAnalysisHydropower

In [ ]:
paths = json.load(open("../paths.json"))

path_data_nexuse_simulations = Path(paths["path_data_nexuse_simulations"])
path_figs = Path(paths["path_figs"])

In [ ]:
date_range_2050_hourly = pd.date_range("2050-01-01", "2051-01-01", freq="1H", inclusive="left")

In [ ]:
cm = 1/2.54

In [ ]:
paths_simulations_ref_renewable = sorted(list((path_data_nexuse_simulations / "centiv_v47_ref_renewable_target").glob("*")))
simulation_ref_renewable_names = ([int(path.stem.split("_")[-1]) for path in paths_simulations_ref_renewable])

paths_simulations_high_renewable = sorted(list((path_data_nexuse_simulations / "centiv_v47_high_renewable_target").glob("*")))
simulation_high_renewable_names = ([int(path.stem.split("_")[-1]) for path in paths_simulations_high_renewable])

# Load data

## Total capacity in 2050

In [ ]:
dfs_cap_ref_renewable = [pd.read_csv(path / "resultPostProcess" / "Cap_CH_centiv_distiv.csv", index_col=0)[["2050"]] 
                        for path in paths_simulations_ref_renewable]

dfs_cap_high_renewable = [pd.read_csv(path / "resultPostProcess" / "Cap_CH_centiv_distiv.csv", index_col=0)[["2050"]] 
                           for path in paths_simulations_high_renewable]

In [ ]:
df_cap_ref_renewable = pd.concat(dfs_cap_ref_renewable, axis=1)
df_cap_ref_renewable.columns = simulation_ref_renewable_names
df_cap_ref_renewable.head()

In [ ]:
df_cap_high_renewable = pd.concat(dfs_cap_high_renewable, axis=1)
df_cap_high_renewable.columns = simulation_high_renewable_names
df_cap_high_renewable.head()

## Total generation in 2050

In [ ]:
dfs_gens_ref_renewable = []
for path in paths_simulations_ref_renewable:
    df_gens_ref_renewable = pd.read_csv(path / "resultPostProcess" / "Gen_CH_2050_c.csv", index_col=0) 
    df_gens_ref_renewable.index = date_range_2050_hourly
    dfs_gens_ref_renewable.append(df_gens_ref_renewable)

dfs_gens_high_renewable = []
for path in paths_simulations_high_renewable:
    df_gens_high_renewable = pd.read_csv(path / "resultPostProcess" / "Gen_CH_2050_c.csv", index_col=0) 
    df_gens_high_renewable.index = date_range_2050_hourly
    dfs_gens_high_renewable.append(df_gens_high_renewable)

In [ ]:
df_gen_ref_renewable = pd.concat(dfs_gens_ref_renewable, axis=1, keys=simulation_ref_renewable_names)
df_gen_high_renewable = pd.concat(dfs_gens_high_renewable, axis=1, keys=simulation_high_renewable_names)

In [ ]:
df_gen_ref_renewable.head()

In [ ]:
df_annual_ror_gen_ref_renewable = df_gen_ref_renewable.loc[:, (slice(None), "RoR")].sum().droplevel(1)
df_annual_imports_ref_renewable = df_gen_ref_renewable.loc[:, (slice(None), "Imports")].sum().droplevel(1)
df_annual_exports_ref_renewable = df_gen_ref_renewable.loc[:, (slice(None), "Exports")].sum().droplevel(1)

In [ ]:
df_annual_ror_gen_high_renewable = df_gen_high_renewable.loc[:, (slice(None), "RoR")].sum().droplevel(1)
df_annual_imports_high_renewable = df_gen_high_renewable.loc[:, (slice(None), "Imports")].sum().droplevel(1)
df_annual_exports_high_renewable = df_gen_high_renewable.loc[:, (slice(None), "Exports")].sum().droplevel(1)

In [ ]:
df_monthly_ror_gen_ref_renewable = df_gen_ref_renewable.loc[:, (slice(None), "RoR")].groupby(pd.Grouper(freq="M")).sum().droplevel(1, axis=1)
df_monthly_ror_gen_ref_renewable.index = pd.date_range("2050-01-01", "2051-01-01", freq="M", inclusive="left").strftime("%b")

df_monthly_ror_gen_high_renewable = df_gen_high_renewable.loc[:, (slice(None), "RoR")].groupby(pd.Grouper(freq="M")).sum().droplevel(1, axis=1)
df_monthly_ror_gen_high_renewable.index = pd.date_range("2050-01-01", "2051-01-01", freq="M", inclusive="left").strftime("%b")

In [ ]:
winter_months = [1, 2, 3, 10, 11, 12]

df_monthly_imports_ref_renewable = df_gen_ref_renewable.loc[:, (slice(None), "Imports")].groupby(pd.Grouper(freq='M')).sum().droplevel(1, axis=1)
df_monthly_exports_ref_renewable = df_gen_ref_renewable.loc[:, (slice(None), "Exports")].groupby(pd.Grouper(freq='M')).sum().droplevel(1, axis=1)

df_winter_imports_ref_renewable = df_monthly_imports_ref_renewable[df_monthly_imports_ref_renewable.index.month.isin(winter_months)].sum(axis=0)
df_winter_exports_ref_renewable = df_monthly_exports_ref_renewable[df_monthly_exports_ref_renewable.index.month.isin(winter_months)].sum(axis=0)

In [ ]:
df_monthly_imports_high_renewable = df_gen_high_renewable.loc[:, (slice(None), "Imports")].groupby(pd.Grouper(freq='M')).sum().droplevel(1, axis=1)
df_monthly_exports_high_renewable = df_gen_high_renewable.loc[:, (slice(None), "Exports")].groupby(pd.Grouper(freq='M')).sum().droplevel(1, axis=1)

df_winter_imports_high_renewable = df_monthly_imports_high_renewable[df_monthly_imports_high_renewable.index.month.isin(winter_months)].sum(axis=0)
df_winter_exports_high_renewable = df_monthly_exports_high_renewable[df_monthly_exports_high_renewable.index.month.isin(winter_months)].sum(axis=0)

In [ ]:
df_pumped_storage_ref_renewable = df_gen_ref_renewable.loc[:, (slice(None), df_gen_ref_renewable.columns.get_level_values(1).str.startswith("Pump-Open"))].groupby(pd.Grouper(freq='M')).sum() * 1e-3
df_pumped_storage_high_renewable = df_gen_high_renewable.loc[:, (slice(None), df_gen_high_renewable.columns.get_level_values(1).str.startswith("Pump-Open"))].groupby(pd.Grouper(freq='M')).sum() * 1e-3

In [ ]:
df_annual_wind_ref_renewable = df_gen_ref_renewable.loc[:, (slice(None), "Wind-Total")].sum().droplevel(1)
df_annual_wind_high_renewable = df_gen_high_renewable.loc[:, (slice(None), "Wind-Total")].sum().droplevel(1)

In [ ]:
df_annual_net_generation_ref_renewable = df_gen_ref_renewable.groupby(pd.Grouper(freq='Y')).sum().groupby(level=0, axis=1).sum().sum()
df_annual_net_generation_high_renewable = df_gen_high_renewable.groupby(pd.Grouper(freq='Y')).sum().groupby(level=0, axis=1).sum().sum()

In [ ]:
df_annual_domestic_generation_ref_renewable = df_gen_ref_renewable.loc[:, (slice(None), slice("Dam", "Pump-Closed"))].groupby(pd.Grouper(freq='Y')).sum().groupby(level=0, axis=1).sum().sum()
df_annual_domestic_generation_high_renewable = df_gen_high_renewable.loc[:, (slice(None), slice("Dam", "Pump-Closed"))].groupby(pd.Grouper(freq='Y')).sum().groupby(level=0, axis=1).sum().sum()

## Electricity prices

### Annual

In [ ]:
dfs_elec_price_ref_renewable = []

for path in paths_simulations_ref_renewable:
    path_file = list((path / "resultPostProcess").glob("**/national_elecprice_annual_c.csv"))[0]
    df_elec_price_ref_renewable = pd.read_csv(path_file, index_col=0) 
    dfs_elec_price_ref_renewable.append(df_elec_price_ref_renewable)

dfs_elec_price_high_renewable = []

for path in paths_simulations_high_renewable:
    path_file = list((path / "resultPostProcess").glob("**/national_elecprice_annual_c.csv"))[0]
    df_elec_price_high_renewable = pd.read_csv(path_file, index_col=0) 
    dfs_elec_price_high_renewable.append(df_elec_price_high_renewable)

In [ ]:
df_elec_price_ref_renewable = pd.concat(dfs_elec_price_ref_renewable, axis=1)
df_elec_price_ref_renewable.columns = simulation_ref_renewable_names
df_elec_price_ref_renewable.head()

In [ ]:
df_elec_price_high_renewable = pd.concat(dfs_elec_price_high_renewable, axis=1)
df_elec_price_high_renewable.columns = simulation_high_renewable_names
df_elec_price_high_renewable.head()

### Monthly

In [ ]:
dfs_elec_price_monthly_ref_renewable = []

for path in paths_simulations_ref_renewable:
    path_file = list((path / "resultPostProcess").glob("**/national_elecprice_monthly_c_2050.csv"))[0]
    df_elec_price_monthly_ref_renewable = pd.read_csv(path_file, index_col=0)["Switzerland"] 
    dfs_elec_price_monthly_ref_renewable.append(df_elec_price_monthly_ref_renewable)

dfs_elec_price_monthly_high_renewable = []

for path in paths_simulations_high_renewable:
    path_file = list((path / "resultPostProcess").glob("**/national_elecprice_monthly_c_2050.csv"))[0]
    df_elec_price_monthly_high_renewable = pd.read_csv(path_file, index_col=0)["Switzerland"] 
    dfs_elec_price_monthly_high_renewable.append(df_elec_price_monthly_high_renewable)

In [ ]:
df_elec_price_monthly_ref_renewable = pd.concat(dfs_elec_price_monthly_ref_renewable, axis=1)
df_elec_price_monthly_ref_renewable.columns = simulation_ref_renewable_names
df_elec_price_monthly_ref_renewable.head()

In [ ]:
df_elec_price_monthly_high_renewable = pd.concat(dfs_elec_price_monthly_high_renewable, axis=1)
df_elec_price_monthly_high_renewable.columns = simulation_high_renewable_names
df_elec_price_monthly_high_renewable.head()

In [ ]:
dfs_elec_price_monthly_ref_renewable_all = []

for path in paths_simulations_ref_renewable:
    path_file = list((path / "resultPostProcess").glob("**/national_elecprice_monthly_c_2050.csv"))[0]
    df_elec_price_monthly_ref_renewable_all = pd.read_csv(path_file, index_col=0)
    dfs_elec_price_monthly_ref_renewable_all.append(df_elec_price_monthly_ref_renewable_all)

df_elec_price_monthly_ref_renewable_all = (pd.concat(dfs_elec_price_monthly_ref_renewable_all, 
                                                   axis=1, keys=simulation_ref_renewable_names)
                                          .swaplevel(0, 1, axis=1)
                                          .sort_index(axis=1, level=0))

dfs_elec_price_monthly_high_renewable_all = []

for path in paths_simulations_high_renewable:
    path_file = list((path / "resultPostProcess").glob("**/national_elecprice_monthly_c_2050.csv"))[0]
    df_elec_price_monthly_high_renewable_all = pd.read_csv(path_file, index_col=0)
    dfs_elec_price_monthly_high_renewable_all.append(df_elec_price_monthly_high_renewable_all)

df_elec_price_monthly_high_renewable_all = (pd.concat(dfs_elec_price_monthly_high_renewable_all, 
                                                      axis=1, keys=simulation_high_renewable_names)
                                             .swaplevel(0, 1, axis=1)
                                             .sort_index(axis=1, level=0))

### Seasonal

In [ ]:
df_elec_price_summer_winter_high_renewable = df_elec_price_monthly_high_renewable.copy()
df_elec_price_summer_winter_high_renewable.index = ["Winter" if i < 4 or i > 9 else "Summer" for i in range(1, 13)]
df_elec_price_summer_winter_high_renewable = df_elec_price_summer_winter_high_renewable.groupby(df_elec_price_summer_winter_high_renewable.index).agg("mean")

df_elec_price_summer_winter_ref_renewable = df_elec_price_monthly_ref_renewable.copy()
df_elec_price_summer_winter_ref_renewable.index = ["Winter" if i < 4 or i > 9 else "Summer" for i in range(1, 13)]
df_elec_price_summer_winter_ref_renewable = df_elec_price_summer_winter_ref_renewable.groupby(df_elec_price_summer_winter_ref_renewable.index).agg("mean")

In [ ]:
seasons = {"DJF": [12, 1, 2], "MAM": [3, 4, 5], "JJA": [6, 7, 8], "SON": [9, 10, 11]}
seasons = {i: season for season in seasons for i in seasons[season]}

df_elec_price_seasonal_high_renewable = df_elec_price_monthly_high_renewable.copy()
df_elec_price_seasonal_high_renewable.index = [seasons[i] for i in range(1, 13)]
df_elec_price_seasonal_high_renewable = df_elec_price_seasonal_high_renewable.groupby(df_elec_price_seasonal_high_renewable.index).agg("mean")
df_elec_price_seasonal_high_renewable = df_elec_price_seasonal_high_renewable.loc[["DJF", "MAM", "JJA", "SON"]]


df_elec_price_seasonal_ref_renewable = df_elec_price_monthly_ref_renewable.copy()
df_elec_price_seasonal_ref_renewable.index = [seasons[i] for i in range(1, 13)]
df_elec_price_seasonal_ref_renewable = df_elec_price_seasonal_ref_renewable.groupby(df_elec_price_seasonal_ref_renewable.index).agg("mean")
df_elec_price_seasonal_ref_renewable = df_elec_price_seasonal_ref_renewable.loc[["DJF", "MAM", "JJA", "SON"]]

In [ ]:
df_elec_price_seasonal_ref_renewable

## Imports/Exports prices

In [ ]:
dfs_elec_price_hourly_ref_renewable = []

for path in paths_simulations_ref_renewable:
    path_file = list((path / "resultPostProcess").glob("**/national_elecprice_hourly_c_2050.csv"))[0]
    df_elec_price_hourly_ref_renewable = pd.read_csv(path_file, index_col=0)["Switzerland"] 
    dfs_elec_price_hourly_ref_renewable.append(df_elec_price_hourly_ref_renewable)

dfs_elec_price_hourly_high_renewable = []

for path in paths_simulations_high_renewable:
    path_file = list((path / "resultPostProcess").glob("**/national_elecprice_hourly_c_2050.csv"))[0]
    df_elec_price_hourly_high_renewable = pd.read_csv(path_file, index_col=0)["Switzerland"] 
    dfs_elec_price_hourly_high_renewable.append(df_elec_price_hourly_high_renewable)

In [ ]:
df_elec_price_hourly_ref_renewable = pd.concat(dfs_elec_price_hourly_ref_renewable, axis=1)
df_elec_price_hourly_ref_renewable.columns = simulation_ref_renewable_names
df_elec_price_hourly_ref_renewable.head()

In [ ]:
df_elec_price_hourly_high_renewable = pd.concat(dfs_elec_price_hourly_high_renewable, axis=1)
df_elec_price_hourly_high_renewable.columns = simulation_high_renewable_names
df_elec_price_hourly_high_renewable.head()

In [ ]:
df_ratio_imports_elecprice_ref_renewable = (df_gen_ref_renewable.loc[:, (slice(None), "Imports")].droplevel(1, axis=1) / df_elec_price_hourly_ref_renewable.set_index(df_gen_ref_renewable.index)).replace([np.inf, -np.inf], np.nan)
df_ratio_exports_elecprice_ref_renewable = ((-df_gen_ref_renewable.loc[:, (slice(None), "Exports")]).droplevel(1, axis=1) / df_elec_price_hourly_ref_renewable.set_index(df_gen_ref_renewable.index)).replace([np.inf, -np.inf], np.nan)

df_monthly_mean_ratio_imports_elecprice_ref_renewable = df_ratio_imports_elecprice_ref_renewable.groupby(pd.Grouper(freq='M')).agg(lambda x: x.dropna().mean())
df_monthly_mean_ratio_exports_elecprice_ref_renewable = df_ratio_exports_elecprice_ref_renewable.groupby(pd.Grouper(freq='M')).agg(lambda x: x.dropna().mean())

In [ ]:
df_ratio_imports_elecprice_high_renewable = (df_gen_high_renewable.loc[:, (slice(None), "Imports")].droplevel(1, axis=1) / df_elec_price_hourly_high_renewable.set_index(df_gen_ref_renewable.index)).replace([np.inf, -np.inf], np.nan)
df_ratio_exports_elecprice_high_renewable = ((-df_gen_high_renewable.loc[:, (slice(None), "Exports")]).droplevel(1, axis=1) / df_elec_price_hourly_high_renewable.set_index(df_gen_ref_renewable.index)).replace([np.inf, -np.inf], np.nan)

df_monthly_mean_ratio_imports_elecprice_high_renewable = df_ratio_imports_elecprice_high_renewable.groupby(pd.Grouper(freq='M')).agg(lambda x: x.dropna().mean())
df_monthly_mean_ratio_exports_elecprice_high_renewable = df_ratio_exports_elecprice_high_renewable.groupby(pd.Grouper(freq='M')).agg(lambda x: x.dropna().mean())

# Paper plots

In [ ]:
file_format = "pdf"

In [ ]:
fontsize_ticks = 6
fontsize_labels = 7
fontsize_title = 8

In [ ]:
blue = sns.color_palette("colorblind")[0]
orange = sns.color_palette("colorblind")[1]
green = sns.color_palette("colorblind")[2]
red = sns.color_palette("colorblind")[3]

In [ ]:
df_annual_ror_gen_ref_renewable_twh = (df_annual_ror_gen_ref_renewable * 1e-6).to_numpy()
df_annual_ror_gen_high_renewable_twh = (df_annual_ror_gen_high_renewable * 1e-6).to_numpy()
df_annual_ror_gen_combined = np.sort(np.concatenate([df_annual_ror_gen_ref_renewable_twh, df_annual_ror_gen_high_renewable_twh]))

df_monthly_ror_gen_ref_renewable_twh = (df_monthly_ror_gen_ref_renewable * 1e-6)
df_monthly_ror_gen_high_renewable_twh = (df_monthly_ror_gen_high_renewable * 1e-6)

In [ ]:
df_annual_ror_gen_ref_renewable_anomaly = df_annual_ror_gen_ref_renewable_twh - df_annual_ror_gen_ref_renewable_twh.mean()
df_annual_ror_gen_high_renewable_anomaly = df_annual_ror_gen_high_renewable_twh - df_annual_ror_gen_high_renewable_twh.mean()

In [ ]:
df_monthly_ror_gen_ref_renewable_anomaly = df_monthly_ror_gen_ref_renewable_twh.sub(df_monthly_ror_gen_ref_renewable_twh.mean(axis=1), axis=0)
df_monthly_ror_gen_high_renewable_anomaly = df_monthly_ror_gen_high_renewable_twh.sub(df_monthly_ror_gen_high_renewable_twh.mean(axis=1), axis=0)

In [ ]:
df_elec_price_monthly_ref_renewable_anomaly = df_elec_price_monthly_ref_renewable.sub(df_elec_price_monthly_ref_renewable.mean(axis=1), axis=0)

## Fig 5 - Electricity prices

In [ ]:
df_elec_price_monthly_ref_renewable.index = pd.date_range("2050-01-01", "2051-01-01", freq="M", inclusive="left").strftime("%b")
df_elec_price_monthly_high_renewable.index = pd.date_range("2050-01-01", "2051-01-01", freq="M", inclusive="left").strftime("%b")

In [ ]:
fig = plt.figure(figsize=(15 * cm, 10 * cm), layout="constrained")
spec = fig.add_gridspec(2, 2, height_ratios=[1, 1])

################################################################################################################

ax0 = fig.add_subplot(spec[0, :])
max_y = np.max([df_elec_price_monthly_ref_renewable.max(), df_elec_price_monthly_high_renewable.max()])
min_y = np.min([df_elec_price_monthly_ref_renewable.min(), df_elec_price_monthly_high_renewable.min()])

df_elec_price_monthly_ref_renewable.plot(ax=ax0, alpha=0.1, color=green, legend=False, rasterized=True)
df_elec_price_monthly_high_renewable.plot(ax=ax0, alpha=0.1, color=orange, legend=False, rasterized=True)

df_elec_price_monthly_ref_renewable.mean(axis=1).rename("mean_ref_renewable").plot(ax=ax0, color=green, label="Reference", legend=True)
df_elec_price_monthly_high_renewable.mean(axis=1).rename("mean_high_renewable").plot(ax=ax0, color=orange, label="High renewable target", legend=True)

ax0.axvline(2, color="black", alpha=0.4, linestyle="--")
ax0.axvline(9, color="black", alpha=0.4, linestyle="--")

ax0.text(x=0.25, y=155, s="Winter", fontsize=fontsize_ticks)
ax0.text(x=10.2, y=155, s="Winter", fontsize=fontsize_ticks)
ax0.text(x=5.1, y=155, s="Summer", fontsize=fontsize_ticks)

ax0.set_ylim([min_y - 5, max_y + 5])
ax0.set_ylabel("Electricity price (in EUR/MWh)", fontsize=fontsize_labels, labelpad=8)

ax0.set_xticks(range(len(df_elec_price_monthly_ref_renewable)))
ax0.set_xticklabels(df_elec_price_monthly_ref_renewable.index)

ax0.tick_params(axis='both', which='major', labelsize=fontsize_ticks)
ax0.tick_params(axis='both', which='minor', labelsize=fontsize_ticks)

ax0.set_title("a", fontweight='bold', loc="left", fontsize=fontsize_title, x=-0.085, y=1.05)

handles, labels = ax0.get_legend_handles_labels()
ax0.legend(handles=handles[-2:], labels=labels[-2:], loc='upper center', fontsize=fontsize_labels)

ax11 = fig.add_subplot(spec[1, :])

sigma_ratios = np.log2(df_elec_price_monthly_ref_renewable.std(axis=1).rename("mean_ref_renewable") / 
                df_elec_price_monthly_high_renewable.std(axis=1).rename("mean_high_renewable"))

sigma_ratios.plot.bar(ax=ax11, color=blue, legend=False)

ax11.set_xticklabels(ax11.get_xticklabels(which="both"), rotation=0)

ax11.set_ylabel(r'Multiplication factor of spread', fontsize=fontsize_labels)
yticks = np.arange(int(sigma_ratios.min()) - 1, int(sigma_ratios.max()) + 1, 1, dtype=float)
ytick_labels = ["x " + str(int(2**n)) if n >= 0 else r"$\div$ " + str(int(2**(-n))) for n in yticks]
ax11.set_yticks(yticks, labels=ytick_labels)

ax11.tick_params(axis='both', which='major', labelsize=fontsize_ticks)
ax11.tick_params(axis='both', which='minor', labelsize=fontsize_ticks)

ax11.set_title("b", fontweight='bold', loc="left", fontsize=fontsize_title, x=-0.085, y=1.05)

plt.savefig(path_figs / "fig_5.pdf", format=file_format, dpi=300, bbox_inches="tight")
plt.show()

## Fig 6 - Wind capacity + net winter imports

In [ ]:
def compute_slopes(
        x_array: np.ndarray,
        y_series: pd.Series
    ) -> Dict:
    X = x_array.reshape(-1, 1)
    coef, _, _, conf_interval = NationalAnalysisHydropower.compute_trend_statsmodel(X, y_series.values)
    dict_coeffs ={
        "name": y_series.name,
        "coef": coef,
        "lower": conf_interval[1][0],
        "upper": conf_interval[1][1],
    }
    
    return dict_coeffs

def compute_error_bars(df_coeffs_scenarios: pd.DataFrame) -> List[float]:
    per_ror_gen_error = [
        [df_coeffs_scenarios["coef"].iloc[0] - df_coeffs_scenarios["lower"].iloc[0], 
         df_coeffs_scenarios["upper"].iloc[0] - df_coeffs_scenarios["coef"].iloc[0]],
        [df_coeffs_scenarios["coef"].iloc[1] - df_coeffs_scenarios["lower"].iloc[1], 
         df_coeffs_scenarios["upper"].iloc[1] - df_coeffs_scenarios["coef"].iloc[1]]
    ]

    return per_ror_gen_error

In [ ]:
columns_df_gen = df_gen_ref_renewable.loc[:, (1991, slice(None))].columns.get_level_values(1).to_list()
columns_df_gen.remove("RoR")
df_slopes_gen = []
for column in columns_df_gen:
    df_slopes_gen.append(
        pd.concat([pd.DataFrame([
            compute_slopes(df_annual_ror_gen_ref_renewable_twh, 
                           (df_gen_ref_renewable.loc[:, (slice(None), column)].sum().droplevel(1) * 1e-6).rename("Reference")
                          ),
            compute_slopes(df_annual_ror_gen_high_renewable_twh, 
                           (df_gen_high_renewable.loc[:, (slice(None), column)].sum().droplevel(1) * 1e-6).rename("High renewable target")
                          )
        ]), pd.Series([column]*2, name="type")], axis=1)
    )
df_slopes_gen = pd.concat(df_slopes_gen).set_index(["type", "name"])
df_slopes_gen.head()

In [ ]:
df_slopes_gen_ref_renewable = df_slopes_gen.loc[(slice(None), "Reference"), :]
df_slopes_gen_ref_renewable.sort_values("coef").head(5)

In [ ]:
df_slopes_gen_high_renewable = df_slopes_gen.loc[(slice(None), "High renewable target"), :]
df_slopes_gen_high_renewable.sort_values("coef").head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(7.5 * cm, 5 * cm))

colors = [green, orange]
alphas = [(0.8,), (0.8,)]
colors_with_alpha = [c + a for c, a in zip(colors, alphas)]

compensation_tech = ["Dam", "PV-alpine", "Imports"]

errors = {tech: compute_error_bars(df_slopes_gen.loc[(tech, slice(None)), :]) for tech in compensation_tech}

x_positions = np.array([0.8, 2.6, 4.4])  # Positions for the two variables
width = 0.4  # Separation for scenarios

# Plot for Scenario 1
ax.errorbar(x_positions - width/2, 
               [df_slopes_gen.loc[(tech, slice(None)), "coef"].iloc[0] for tech in compensation_tech], 
               yerr=np.transpose([errors[tech][0] for tech in compensation_tech]), fmt='o', 
               label='Reference', color=colors_with_alpha[0], ecolor=colors_with_alpha[0], 
            elinewidth=1, linewidth=0, markeredgewidth=0, rasterized=(file_format == "eps"))

# Plot for Scenario 2
ax.errorbar(x_positions + width/2, 
               [df_slopes_gen.loc[(tech, slice(None)), "coef"].iloc[1] for tech in compensation_tech], 
               yerr=np.transpose([errors[tech][1] for tech in compensation_tech]), fmt='o', 
               label='High renewable target', color=colors_with_alpha[1], ecolor=colors_with_alpha[1], 
            elinewidth=1, markeredgewidth=0, rasterized=(file_format == "eps"))

ax.axvline(1.7, color="black", alpha=0.4, linestyle="--")
ax.axvline(3.5, color="black", alpha=0.4, linestyle="--")


ax.set_xticks(x_positions)
ax.set_xticklabels(['Dam Generation\n(in TWh)', 'Alpine PV Generation\n(in TWh)', 'Imports \n(in TWh)'])
ax.set_ylabel('Slope (-)', fontsize=fontsize_labels)
ax.set_xlim(0, 5)

ax.tick_params(axis="both", which="major", labelsize=fontsize_ticks)
ax.tick_params(axis="both", which="minor", labelsize=fontsize_ticks)

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles=handles, labels=labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.18), fontsize=fontsize_labels)

plt.savefig(path_figs / "fig_6.pdf", format=file_format, dpi=300, bbox_inches="tight")
plt.show()

## Sup Fig 5 - Comparison of the systems generated by the two scenarios (average generation of key technologies)

In [ ]:
dark_blue = (0, 0.25098039215686274, 0.49803921568627446)
medium_blue = (0.17058823529411765, 0.5784313725490197, 0.8058823529411765)
light_grey = (0.7803921568627452, 0.7803921568627452, 0.7803921568627452)
tech_categories_colors = [
    dark_blue,
    sns.color_palette("colorblind")[0], 
    medium_blue, 
    sns.color_palette("colorblind")[2], 
    sns.color_palette("colorblind")[1], 
    sns.color_palette("colorblind")[5], 
    sns.color_palette("colorblind")[7],
    sns.color_palette("colorblind")[-1],
    light_grey
]

In [ ]:
fig, ax = plt.subplots(figsize=(15*cm, 5*cm), layout="constrained")
columns = ['Dam', 'RoR', 'Pump-Open', 'WindOn', 'PV-roof', 'PV-alpine', 'Imports','Pump-Open (Load)', 'Exports']

avg_gen_scenarios = pd.concat([(df_gen_ref_renewable.loc[:, (slice(None), columns)].sum().droplevel(0).groupby(level=0).mean() * 1e-6).rename("Reference"),
           (df_gen_high_renewable.loc[:, (slice(None), columns)].sum().droplevel(0).groupby(level=0).mean() * 1e-6).rename("High renewable target")], axis=1)

avg_gen_scenarios = avg_gen_scenarios.loc[columns]

avg_gen_scenarios.T.plot.barh(ax=ax, stacked=True, color=tech_categories_colors)

ax.set_xlabel("Generation (in TWh)", fontsize=fontsize_labels)
ax.tick_params(axis="both", which="major", labelsize=fontsize_ticks)
ax.tick_params(axis="both", which="minor", labelsize=fontsize_ticks)

plt.legend(loc="center right", bbox_to_anchor=(1.4, 0.5), fontsize=fontsize_labels)
plt.savefig(path_figs / "sup_fig_5.pdf", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
avg_gen_scenarios

## Sup Fig 6 - Understanding the effect of weather types on electricity prices

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(10.5*cm, 7.5*cm), layout="constrained")

winter_months = df_elec_price_monthly_ref_renewable.index.tolist()[:3] + df_elec_price_monthly_ref_renewable.index.tolist()[9:]

elec_price_range = (df_elec_price_monthly_ref_renewable.loc[winter_months].min(axis=None) - 20, 
                    df_elec_price_monthly_ref_renewable.loc[winter_months].max(axis=None) + 20)

ror_gen_anomaly_range = (df_monthly_ror_gen_ref_renewable_anomaly.loc[winter_months].min(axis=None) - 0.2, 
                         df_monthly_ror_gen_ref_renewable_anomaly.loc[winter_months].max(axis=None) + 0.1)


ax = ax.flatten()
for i, month in enumerate(winter_months):
    ax[i].set_xlim(*ror_gen_anomaly_range)
    ax[i].set_ylim(*elec_price_range)
    # sns.regplot(x=df_monthly_ror_gen_ref_renewable_anomaly.loc[month], 
    #             y=df_elec_price_monthly_ref_renewable.loc[month].to_numpy(), 
    #             color=blue, 
    #             scatter_kws={"color": blue, "alpha": 0.2}, 
    #             line_kws={"color": blue, "alpha": 1, "lw": 2}, 
    #             truncate=False, 
    #             ax=ax[i],
    #             n_boot=2000,
    #             seed=20)
    ax[i].scatter(x=df_monthly_ror_gen_ref_renewable_anomaly.loc[month], 
                y=df_elec_price_monthly_ref_renewable.loc[month],
                color=blue,
                alpha=0.5)
    
    ax[i].set_xlabel("")
    ax[i].set_title(month, fontsize=fontsize_title)

fig.supxlabel("RoR generation anomaly (in TWh)", fontsize=fontsize_labels)
fig.supylabel("Electricity price (in EUR/MWh)", fontsize=fontsize_labels)

for i in range(len(ax)):
    ax[i].tick_params(axis="both", which="major", labelsize=fontsize_ticks)
    ax[i].tick_params(axis="both", which="minor", labelsize=fontsize_ticks)

plt.savefig(path_figs / "sup_fig_6.pdf", dpi=300, bbox_inches="tight")
plt.show()